<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/master/notebooks/04a_fruta_acrecentamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acrecentamieto de datos para el reconocimiento de frutas

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import os
import glob

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.layers import Add, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical

np.random.seed(1)
tf.random.set_seed(2)

## Conjunto de datos

Vamos a entrenar una red neuronal  convolucional para reconocimiento de frutas usando el conjunto de datos: *Fruits 360*, el cual está compuesto por imágenes a color de $100 \times 100 \times 3$ de 75 frutas distintas. 

Para descargar este conjunto de imágenes simplemente clonamos el repositorio:



In [0]:
!git clone https://github.com/Horea94/Fruit-Images-Dataset.git

La estructura del repositorio clonado:

In [0]:
!ls Fruit-Images-Dataset/

El conjunto de imágenes se encuentra dividido en subconjuntos de entrenamiento y prueba, los cuales están contenidos en los subdirectorios `Training` y `Test`. A su vez, estos subdirectorios contienen un directorio por cada categoría de fruta y dentro de cada uno de ellos están las imágenes correspondientes. Vamos a recopilar cada una de estas imágenes con su correspondiente etiqueta:

In [0]:
# Conjunto de entrenamiento
ruta_ent = 'Fruit-Images-Dataset/Training'
ruta_prueba = 'Fruit-Images-Dataset/Test'
dirs_ent = os.listdir(ruta_ent)
dirs_prueba = os.listdir(ruta_prueba)

etiquetas = dirs_ent
ind_a_str = {i:s for i,s in enumerate(dirs_ent)}
str_a_ind = {s:i for i,s in enumerate(dirs_ent)}

img_ent = []
etiq_ent = []
cat_ent = []
for e in etiquetas:
  for f in os.listdir(ruta_ent + '/' + e):
    img_ent.append(ruta_ent + '/' + e + '/' + f)
    etiq_ent.append(e)
    cat_ent.append(str_a_ind[e])

# Conjunto de prueba
ruta_prueba = 'Fruit-Images-Dataset/Test'
dirs_prueba = os.listdir(ruta_prueba)

img_prueba = []
etiq_prueba = []
cat_prueba = []
for e in etiquetas:
  for f in os.listdir(ruta_prueba + '/' + e):
    img_prueba.append(ruta_prueba + '/' + e + '/' + f)
    etiq_prueba.append(e)
    cat_prueba.append(str_a_ind[e])

Ahora inspeccionémos el número de imágenes de entrenamiento:

In [0]:
n_ej = len(img_ent)
print(n_ej)

Ahora veámos cuántas clases tenemos:

In [0]:
n_clases = len(dirs_ent)
print(n_clases)

Ya descargadas las imágenes, podemos visualizar algunas:

In [0]:
rand_s = np.random.randint(n_ej, size = 9)
for i in range(9):
  img = plt.imread(img_ent[rand_s[i]])
  plt.subplot(3, 3, i + 1)
  plt.imshow(img)
  plt.axis('off')

Algunos ejemplos de rutas y sus correspondientes etiquetas e idenficadores:

In [0]:
df = pd.DataFrame(list(zip(img_ent, etiq_ent, cat_ent)), columns =['Archivo', 'Etiqueta', u'Categoría'])
df.sample(10)

Hacemos los mismo para imágenes del conjunto de prueba:

In [0]:
df_prueba = pd.DataFrame(list(zip(img_prueba, etiq_prueba, cat_prueba)), columns =['Archivo', 'Etiqueta', u'Categoría'])
df.sample(10)

## Partición del conjunto de entrenamiento
Ahora vamos a dividir el conjunto de entrenamiento disponible en entrenamiento y validación para monitorear métricas y pérdidas en estos 2 subconjunto:

In [0]:
n_ent = int(np.floor(df.shape[0] * 0.8))
perm = np.random.permutation(df.shape[0])
df_ent = df.iloc[perm[:n_ent]]
df_val = df.iloc[perm[n_ent:]]

## Lectura y preprocesamiento de imágenes
Generamos funciones para leer y preprocesar nuestras imágenes de entrenamiento:

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from timeit import default_timer as timer

class ImGen:
  def __init__(self):
    self.gen_ent = ImageDataGenerator(rescale=1./255,
                                      rotation_range=90,
                                      horizontal_flip=True)
            
  def __call__(self):
    return self.gen_ent.flow_from_dataframe(dataframe=df_ent, 
                                            directory="./", 
                                            x_col=u'Archivo', 
                                            y_col=u'Etiqueta',
                                            target_size=(64,64),
                                            batch_size=64, 
                                            class_mode="sparse")
    
ent_ds = tf.data.Dataset.from_generator(ImGen(),  
                                        output_types=(tf.float32, tf.float32))

start = timer()
for i, (x, y_true) in enumerate(ent_ds, start=1):
    plt.subplot(2, 5, i)
    plt.imshow(x[0])
    plt.axis('off')
    if i == 10:
        break

end = timer()
print(end - start) 

De forma alternativa, podemos usar la función de mapeo del API de datos de Tensorflow: 

In [0]:
def procesa_imagen(ruta, categoria, imsize=(64,64)):
    imagen = tf.io.read_file(ruta)
    imagen = tf.image.decode_jpeg(imagen, channels=3)
    imagen = tf.image.resize(imagen, imsize)
    imagen /= 255.0
    
    imagen = tf.image.random_flip_left_right(imagen)
    imagen = tf.image.random_flip_up_down(imagen)
    imagen = tf.image.random_crop(imagen, size=[64, 64, 3])
    imagen = tf.image.rot90(imagen, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    imagen = tf.clip_by_value(imagen, 0, 1)
    
    return imagen, categoria

ent_ds = tf.data.Dataset.from_tensor_slices((df_ent['Archivo'], df_ent['Categoría']))
ent_ds = ent_ds.shuffle(n_ej)
ent_ds = ent_ds.map(procesa_imagen, num_parallel_calls=8)
ent_ds = ent_ds.batch(64)
ent_ds = ent_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

start = timer()
for i, (x, y_true) in enumerate(ent_ds, start=1):
    plt.subplot(2, 5, i)
    plt.imshow(x[0])
    plt.axis('off')
    if i == 10:
        break
end = timer()
print(end - start) 

Otra estrategia es serializar los ejemplos y guardarlos como [TFRecords](https://www.tensorflow.org/tutorials/load_data/tfrecord):

In [0]:
def valor_byte(valor):
  if isinstance(valor, type(tf.constant(0))):
    valor = valor.numpy() 
  
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[valor]))

def valor_int64(valor):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[valor]))

def ejemplo_imagen(ruta, etiqueta):
  imagen = tf.io.read_file(ruta)
  imagen = tf.image.decode_jpeg(imagen)
  imagen = tf.image.resize(imagen, (64,64))
  imagen /= 255.0
  cadena_img = tf.io.serialize_tensor(imagen)
  tam_img = imagen.shape

  caract = {
        'height': valor_int64(tam_img[0]),
        'width': valor_int64(tam_img[1]),
        'depth': valor_int64(tam_img[2]),
        'label': valor_int64(etiqueta),
        'image_raw': valor_byte(cadena_img),
      }

  return tf.train.Example(features=tf.train.Features(feature=caract))

tfr_ent = 'ent_images.tfrecords'
with tf.io.TFRecordWriter(tfr_ent) as writer:
  for ruta,etiqueta in zip(df_ent['Archivo'], df_ent['Categoría']):
    tf_ej = ejemplo_imagen(ruta, etiqueta)
    writer.write(tf_ej.SerializeToString())

tfr_val = 'val_images.tfrecords'
with tf.io.TFRecordWriter(tfr_val) as writer:
  for ruta,etiqueta in zip(df_val['Archivo'], df_val['Categoría']):
    tf_ej = ejemplo_imagen(ruta, etiqueta)
    writer.write(tf_ej.SerializeToString())

tfr_prueba = 'prueba_images.tfrecords'
with tf.io.TFRecordWriter(tfr_prueba) as writer:
  for ruta,etiqueta in zip(df_prueba['Archivo'], df_prueba['Categoría']):
    tf_ej = ejemplo_imagen(ruta, etiqueta)
    writer.write(tf_ej.SerializeToString())

Posteriormente, podemos leerlos como un `tf.data.Dataset` usando `tf.data.TFRecordDataset`. Para ello es necesario definir una función para deserializar cada ejemplo:

In [0]:
def imagen_tfr(ej_proto):
  caract = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
  }

  ej = tf.io.parse_single_example(ej_proto, caract)
  imagen = tf.io.parse_tensor(ej['image_raw'], out_type = float)
  img_shape = [ej['height'], ej['width'], ej['depth']]
  imagen = tf.reshape(imagen, img_shape)

  imagen = tf.image.random_flip_left_right(imagen)
  imagen = tf.image.random_flip_up_down(imagen)
  imagen = tf.image.random_crop(imagen, size=[64, 64, 3])
  imagen = tf.image.rot90(imagen, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
  imagen = tf.clip_by_value(imagen, 0, 1)

  return imagen, ej['label']

ent_ds = tf.data.TFRecordDataset('ent_images.tfrecords', buffer_size = 100, num_parallel_reads=8)
ent_ds = ent_ds.shuffle(n_ej)
ent_ds = ent_ds.map(imagen_tfr)
ent_ds = ent_ds.batch(64)
ent_ds = ent_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

start = timer()
for i, (x, y_true) in enumerate(ent_ds, start=1):
    plt.subplot(2, 5, i)
    plt.imshow(x[0])
    plt.axis('off')
    if i == 10:
        break

end = timer()
print(end - start) 

Asimismo, definimos la función para cargar las imágenes de validación: 

In [0]:
def carga_una_imagen(ej_proto):
  caract = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
  }

  ej = tf.io.parse_single_example(ej_proto, caract)
  imagen = tf.io.parse_tensor(ej['image_raw'], out_type = float)
  img_shape = [ej['height'], ej['width'], ej['depth']]
  imagen = tf.reshape(imagen, img_shape)

  return imagen, ej['label']
  
val_ds = tf.data.TFRecordDataset('val_images.tfrecords', buffer_size = 100, num_parallel_reads=8)
val_ds = val_ds.map(carga_una_imagen)
val_ds = val_ds.batch(1)

prueba_ds = tf.data.TFRecordDataset('val_images.tfrecords', buffer_size = 100, num_parallel_reads=8)
prueba_ds = prueba_ds.map(carga_una_imagen)
prueba_ds = prueba_ds.batch(1)

## Definición de la red
Ahora vamos a definir una red neuronal convolucional simple con bloques de tipo residual: 

In [0]:
class CNN(tf.keras.Model):
  def __init__(self):
    super(CNN, self).__init__()
    self.zp01 = ZeroPadding2D((3, 3))
    self.conv01 = Conv2D(64, (7, 7), strides=(2, 2))
    self.bn01 = BatchNormalization(axis = 3)
    self.act01 = Activation('relu')
    self.pool01 = MaxPooling2D((3, 3), strides=(2, 2))

    self.conv11 = Conv2D(64, kernel_size = (1, 1), strides = (1,1))
    self.bn11 = BatchNormalization(axis = 3)
    self.act11 = Activation('relu')
    self.conv12 = Conv2D(64, kernel_size = (3, 3), strides = (1,1), padding = 'same')
    self.bn12 = BatchNormalization(axis = 3)
    self.act12 = Activation('relu')
    self.conv13 = Conv2D(256, kernel_size = (1, 1), strides = (1,1), padding = 'valid')
    self.bn13 = BatchNormalization(axis = 3)
    self.conv14 = Conv2D(256, kernel_size = (1, 1), strides = (1,1), padding = 'valid')
    self.bn14 = BatchNormalization(axis = 3)
    self.add1 = Add()
    self.act13 = Activation('relu')
    

    self.conv21 = Conv2D(64, kernel_size = (1, 1), strides = (1,1), padding = 'valid')
    self.bn21 = BatchNormalization(axis = 3)
    self.act21 = Activation('relu')
    self.conv22 = Conv2D(64, kernel_size = (3, 3), strides = (1,1), padding = 'same')
    self.bn22 = BatchNormalization(axis = 3)
    self.act22 = Activation('relu')
    self.conv23 = Conv2D(256, kernel_size = (1, 1), strides = (1,1), padding = 'valid')
    self.bn23 = BatchNormalization(axis = 3)
    self.add2 = Add()
    self.act23 = Activation('relu')
    
    self.conv31 = Conv2D(64, kernel_size = (1, 1), strides = (1,1), padding = 'valid')
    self.bn31 = BatchNormalization(axis = 3)
    self.act31 = Activation('relu')
    self.conv32 = Conv2D(64, kernel_size = (3, 3), strides = (1,1), padding = 'same')
    self.bn32 = BatchNormalization(axis = 3)
    self.act32 = Activation('relu')
    self.conv33 = Conv2D(256, kernel_size = (1, 1), strides = (1,1), padding = 'valid')
    self.bn33 = BatchNormalization(axis = 3)
    self.add3 = Add()
    self.act33 = Activation('relu')
    
    self.ap = AveragePooling2D((2,2))
    self.flat = Flatten()
    
    self.dense = Dense(512, activation='relu', kernel_regularizer='l2', 
                       bias_regularizer='l2')
    self.dp = Dropout(0.7)
    self.sm = Dense(n_clases, activation='softmax', kernel_regularizer='l2', 
                    bias_regularizer='l2')

  def call(self, x): 
    # etapa 1
    x 
    x = self.zp01(x)
    x = self.conv01(x)
    x = self.bn01(x)
    x = self.act01(x)
    x = self.pool01(x)
    
    # etapa 2: bloque convolucional
    x_s = x     
    x = self.conv11(x)
    x = self.bn11(x)
    x = self.act11(x)
    x = self.conv12(x)
    x = self.bn12(x)
    x = self.act12(x)
    x = self.conv13(x)
    x = self.bn13(x)
    x_s = self.conv14(x_s)
    x_s = self.bn14(x_s)
    x = self.add1([x, x_s])
    x = self.act13(x)

    # etapa 3: primer bloque identidad
    x_s = x     
    x = self.conv21(x)
    x = self.bn21(x)
    x = self.act21(x)
    x = self.conv22(x)
    x = self.bn22(x)
    x = self.act22(x)
    x = self.conv23(x)
    x = self.bn23(x)
    x = self.add2([x, x_s])
    x = self.act23(x)
    
    # etapa 4: segundo bloque identidad
    x_s = x     
    x = self.conv31(x)
    x = self.bn31(x)
    x = self.act31(x)
    x = self.conv32(x)
    x = self.bn32(x)
    x = self.act32(x)
    x = self.conv33(x)
    x = self.bn33(x)
    x = self.add3([x, x_s])
    x = self.act33(x)
  
    # etapa 5: bloque de clasificación
    x = self.ap(x)
    x = self.flat(x)
    x = self.dense(x)
    x = self.dp(x)
    x = self.sm(x)
    
    return x

Construimos nuestra red y visualizamos sus características:

In [0]:
modelo = CNN()
modelo.build(input_shape=(None, 64, 64 , 3))
modelo.summary()

Ahora definimos las funciones para realizar la propagación hacia adelante y la retropropagación en los ejemplos de entrenamiento y la propagación hacia adelante para los ejemplos de validación y de prueba.

In [0]:
@tf.function
def paso_ent(x, y, modelo, fn_perdida, optimizador):
    with tf.GradientTape() as cinta:
        y_pred = modelo(x)
        perdida = fn_perdida(y, y_pred)
    gradientes = cinta.gradient(perdida, modelo.trainable_variables)
    optimizador.apply_gradients(zip(gradientes, modelo.trainable_variables))
    return y_pred
  
@tf.function
def paso_prueba(imagen, modelo):
  return modelo(imagen)

## Entrenamiento
Con las funciones definidas anteriormente entrenamos nuestra red optimizando la función de entropía cruzada categórica:

In [0]:
import tensorflow as tf
fn_perdida = tf.keras.losses.SparseCategoricalCrossentropy()
optimizador = tf.keras.optimizers.Adam(learning_rate=1e-3)

perdida_ent = tf.keras.metrics.SparseCategoricalCrossentropy()
exactitud_ent = tf.keras.metrics.SparseCategoricalAccuracy()

perdida_val = tf.keras.metrics.SparseCategoricalCrossentropy()
exactitud_val = tf.keras.metrics.SparseCategoricalAccuracy()

hist_perdida_ent = []
hist_exactitud_ent = []

hist_perdida_val = []
hist_exactitud_val = []

for epoca in range(5):
    for paso, (x, y) in enumerate(ent_ds):
        y_pred = paso_ent(x, y,  modelo, fn_perdida, optimizador)
        perdida_ent(y, y_pred)
        exactitud_ent(y, y_pred)
        
    for (x, y) in val_ds:
        y_pred = paso_prueba(x, modelo)
        perdida_val(y, y_pred)
        exactitud_val(y, y_pred)
    
    perdida_ent_res = perdida_ent.result().numpy()
    exactitud_ent_res = exactitud_ent.result().numpy() * 100
    perdida_ent.reset_states()
    exactitud_ent.reset_states()
    hist_perdida_ent.append(perdida_ent_res)
    hist_exactitud_ent.append(exactitud_ent_res)

    perdida_val_res = perdida_val.result().numpy()
    exactitud_val_res = exactitud_val.result().numpy() * 100
    perdida_val.reset_states()
    exactitud_val.reset_states()
    hist_perdida_val.append(perdida_val_res)
    hist_exactitud_val.append(exactitud_val_res)
  
    print('E{:2d} CCE(E)={:6.2f}, Exactidud(E)={:6.2f} CCE(V)={:6.2f} Exactitud(V)={:6.2f}'.format(epoca, 
                                                                                                   perdida_ent_res, 
                                                                                                   exactitud_ent_res,
                                                                                                   perdida_val_res, 
                                                                                                   exactitud_val_res))

Visualicemos la evolución de la pérdida de entropía cruzada y la exactitud durante el entrenamiento:

In [0]:
plt.figure(1)
plt.plot(hist_perdida_ent, color='red', label='Entrenamiento')
plt.plot(hist_perdida_val, color='blue', label=u'Validación')
plt.xlabel(u'Época')
plt.ylabel(u'Pérdida')
plt.legend(loc='upper right')

plt.figure(2)
plt.plot(hist_exactitud_ent, color='red', label='Entrenamiento')
plt.plot(hist_exactitud_val, color='blue', label=u'Validación')
plt.xlabel(u'Época')
plt.ylabel(u'Exactitud')
plt.legend(loc='upper right')

plt.show()

Ya entrenado nuestro modelo, procedemos a evaluar su desempeño en el conjunto de imágenes de prueba:

In [0]:
exactitud_prueba = tf.keras.metrics.SparseCategoricalAccuracy()
for i, (x, y) in enumerate(prueba_ds):
  y_pred = paso_prueba(x, modelo)
  exactitud_prueba(y, y_pred)

exactitud_prueba = exactitud_prueba.result().numpy() * 100
print("Exactitud en validación: {:02.2f}%".format(exactitud_prueba))

Finalmente, visualizamos algunos ejemplos de predicciónes hechas por el modelo en el conjunto de prueba.

In [0]:
prueba_ds = prueba_ds.shuffle(len(img_prueba))

fig = plt.figure(figsize=(10, 4))
for i, (x, y) in enumerate(prueba_ds, start=1):
    ax = fig.add_subplot(2, 5, i)
    ax.imshow(x[0])
    y_pred = np.argmax(modelo(x))
    etiq = ind_a_str[y_pred]
    ax.imshow(x[0])
    ax.set_title(ind_a_str[y.numpy()[0]])
    if etiq == ind_a_str[y.numpy()[0]]:
      ax.text(1,8, etiq, color = 'b')
    else:  
      ax.text(1,8, etiq, color = 'r')
    ax.set_xticks([])
    ax.set_yticks([])
    if i == 10:
        break